# Detecting Marine Mammal Calls in MBARI Hydrophone Dataset 

-----------------------

Artash Nath
Founder, MonitorMyOcean.com

--------------------------

In [1]:
# Importing open-source libraries

# AWS Client Data Libraries
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from numba import jit

# Standard Libaries
import os
from pathlib import Path
import soundfile as sf
import json
import datetime
from datetime import datetime, timedelta
from IPython.display import Audio
from tqdm.notebook import trange, tqdm

# Statistics / Analsyis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.patches import Rectangle
import scipy
import pandas as pd
import cv2
import librosa
from scipy.signal import find_peaks
import sklearn
from scipy.ndimage import convolve
import scipy

# Custom Helper Functions

from helper_functions import*

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

----------------
Preliminary Functions and Constants

----------------

In [3]:
# Blue Whale Call Parameters (From MBARI)

BLUE_A = dict(name = 'BlueA',
    freq_range=[(70,90)], duration_secs=25, blur_axis='frequency', num_fft=1024, center=True,  padding_secs=3, num_mels=30)

BLUE_B = dict(name = 'BlueB',
    freq_range=[(40,55)], duration_secs=25, blur_axis='time', num_fft=1024, center=True, padding_secs=2)

BLUE_D = dict(name = 'BlueD',
    freq_range=[(25,75)], duration_secs=7, blur_axis='', num_fft=1024, center=True, padding_secs=2, num_mels=30)

FIN_20 = dict( 
    freq_range=[(10,35)], duration_secs=0, blur_axis='frequency', num_fft=4096, center=False,  padding_secs=3)

# Constants
SAMPLE_RATE = 16000 # Hydrophone File Sample rate
OVERLAP = 0.95 # Overlap when taking precise interst-sample FFTs
IMAGE_SIZE = (224, 224) # Size of sample images being fed into AI models

MBARI_HYDROPHONE_CALIBRATION = 177.9
global MBARI_HYDROPHONE_CALIBRATION

----------------
Downloading Raw Hydrophone Data at 16 kHz from AWS Server

----------------

In [4]:
data_path = r'E:\MBARI DATA'

# Timing parameters
year = 2017
month = 8
bucket = 'pacific-sound-16khz'

# Client to retreive AWS MBARI Raw Acoustic Data
s3 = boto3.client('s3',
    aws_access_key_id='',
    aws_secret_access_key='',
    config=Config(signature_version=UNSIGNED))

filenames = []
for obj in s3.list_objects_v2(Bucket=bucket, Prefix=f'{year:04d}/{month:02d}')['Contents']:
    filenames.append(obj['Key'])
    
s3 = boto3.resource('s3',
    aws_access_key_id='',
    aws_secret_access_key='',
    config=Config(signature_version=UNSIGNED))

# First day in the month    
n = filenames[0][8:]
wav_filename_full = os.path.join(data_path, n)
wav_filename = n
key = f'{year:04d}/{month:02d}/{wav_filename}'
 
# only download if needed
if not Path(wav_filename_full).exists():
    
    print('Downloading') 
    s3.Bucket(bucket).download_file(key, wav_filename_full)
    print('Done')

In [5]:

# Load Blue Whale A-Call Model

model_blue_A = tf.keras.models.load_model(r"C:\Users\vikas\OneDrive\Artash_Python\2023 Test\1 - Whale Detection\ML Models\bluewhale-a-resnet")

config_blue_A = json.load(open(r'C:\Users\vikas\OneDrive\Artash_Python\2023 Test\1 - Whale Detection\ML Models\bluewhale-a-resnet\config.json'))
image_mean_blue_A = np.asarray(config_blue_A["image_mean"])
image_std_blue_A = np.asarray(config_blue_A["image_std"])
print(f"Labels {config_blue_A['classes']}")
print(f"Training image mean: {image_mean_blue_A}")
print(f"Training image std: {image_std_blue_A}")

global model_blue_A
global config_blue_A
global image_mean_blue_A
global image_std_blue_A

Labels ['baf', 'bat']
Training image mean: [0.18747011 0.66276884 0.68202728]
Training image std: [0.03253593 0.01769102 0.01564376]


In [6]:
# Load Humpback Call model and get its score_fn for use in analyzing waveform data at 10kHz:

model_humphback = tf.keras.models.load_model("./ML Models/humphback1")

model_humphback_score = model_humphback.signatures['score']

metadata_fn_humphback = model_humphback.signatures['metadata']
metadata_humphback = metadata_fn_humphback()
print('metadata:')
for k, v in metadata_humphback.items():
    print(f'  {k}: {v}')

global metadata_fn_humphback
global model_humphback_score

C:\ProgramData\Anaconda3\envs\oceansoundscape2\lib\site-packages\keras\layers\core\lambda_layer.py:327: UserWarning: front_end is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(
C:\ProgramData\Anaconda3\envs\oceansoundscape2\lib\site-packages\keras\layers\core\lambda_layer.py:327: UserWarning: humpback_model is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


metadata:
  class_names: [b'Mn']
  context_width_samples: 39124
  input_sample_rate: 10000


In [8]:
model_multipilot = tf.keras.models.load_model('saved_model/multimodel-v1')

----------------

Spectrum Analysis Functions

----------------

In [7]:
def mel_spectrogram(x, sample_rate, num_fft, OVERLAP, freq_range, duration, mammal):
    
    if mammal == 'blue': 
        n_mels = 30
    elif mammal == 'humphback': 
        n_mels = 30
    # Optimum mel spectrogram parameters for classification
    PCEN_GAIN = 0.25
    PCEN_BIAS = 2.0
    PCEN_TIME_CONSTANT = 0.6
    
    hop_length = round(num_fft * (1 - OVERLAP))
    
    stft = librosa.feature.melspectrogram(
                            y=sklearn.preprocessing.minmax_scale(x, feature_range=((-2 ** 31), (2 ** 31))),
                            sr=sample_rate,
                            hop_length=hop_length,
                            power=1,
                            n_mels=n_mels,
                            fmin=freq_range[0],
                            fmax=freq_range[1])
    
    stft_pcen = librosa.pcen(stft * (2 ** 31), sr=sample_rate,
                                                 hop_length=hop_length,
                                                 gain=PCEN_GAIN, bias = PCEN_BIAS,
                                                 time_constant=PCEN_TIME_CONSTANT)
    
    stft_pcen = smooth(stft_pcen, 'frequency')
    
    if mammal == 'blue':
        start = int((duration*2000) / hop_length)
        end = int(2 * (duration*2000) / hop_length)
        #plt.imshow(stft_pcen[:, start:end], origin='lower')
        plt.show()
        im = colorizeDenoise(stft_pcen[:, start:end])[:,:,:3]
        
    elif mammal == 'humphback':
        #plt.imshow(stft_pcen, origin='lower')
        #plt.show()
        im = colorizeDenoise(stft_pcen)[:,:,:3]
    return im, np.percentile(stft_pcen, 50), np.percentile(stft_pcen, 75), np.percentile(stft_pcen, 90)



def mid_freq(a):
    
    weights = np.mean(a[:, :, 0], axis=1)
    weights = weights/np.sum(weights)
    weighted = []
    for c, i in enumerate(weights):
        weighted.append(i*c)
    return np.sum(weighted)

############################################################################################

def predict_a(im):
    image_float = np.asarray(im).astype('float32')
    image_float = image_float / 255.
    image_float = (image_float - image_mean_blue_A) / ( image_std_blue_A + 1.e-9)
    image = np.concatenate([image_float[np.newaxis, :, :]] * 1)
    tensor_out = model_blue_A(image)
    pred = tensor_out[0,1]
    return float(np.array(pred))


def BLUE_Analysis(x_2k, hour, date):
    
    annotations_A = []
    annotations_B = []
    
    sg, f = psd_1sec(x_2k, 2000, MBARI_HYDROPHONE_CALIBRATION) # create calibrated psd
    
    windows_A = find_windows(sg, BLUE_A)
    windows_B = find_windows(sg, BLUE_B)
    print("{} Area-Of-Interests (A-Calls) were found in hour {}".format(len(windows_A), hour))
    print("{} Area-Of-Interests (B-Calls) were found in hour {}".format(len(windows_B), hour))
    
    
    try:
        os.mkdir(r'C:\Users\vikas\OneDrive\Artash_Python\2023 Test\1 - Whale Detection\Annotations\{}'.format(date))
    except:
        pass
    
    directory_A = r'C:\Users\vikas\OneDrive\Artash_Python\2023 Test\1 - Whale Detection\Annotations\{}\Blue_A'.format(date)
    directory_B = r'C:\Users\vikas\OneDrive\Artash_Python\2023 Test\1 - Whale Detection\Annotations\{}\Blue_B'.format(date)
    
    try:
        os.mkdir(directory_A)
        os.mkdir(directory_B)
    except:
        pass

    for t1, t2, f1, f2 in tqdm(windows_A):
        
        try:
            sample_width = BLUE_A['duration_secs']*2000
            subset = x_2k[(t1*2000)-sample_width:(t2*2000)+sample_width]
            im, p50, p75, p90 = mel_spectrogram(subset, 2000, BLUE_A['num_fft'], OVERLAP, [70, 100], BLUE_A['duration_secs'], 'blue')
            p = round(predict_a(im), 3)
            if p > 0.6:
                mid_f = mid_freq(im)
                mid_f = (f2-f1)*(mid_f/224)
                mid_f+=f1
                annotations_A.append([date, hour, t1, 'BLUE_A', p, sample_width, mid_f, p50, p75, p90])
                name = f'{date}T{hour}-{t1}-BlueA'
                plt.imsave(os.path.join(directory_A, name+'.png'), im, origin='lower')
                
            if p < 0.05:
                name = f'{date}T{hour}-{t1}-NoiseA'
                plt.imsave(os.path.join(r'C:\Users\vikas\OneDrive\Artash_Python\2023 Test\1 - Whale Detection\Annotations\Noise', name+'.png'), im, origin='lower')
                
        except:
            pass
        
#################################################################################        
        
    for t1, t2, f1, f2 in tqdm(windows_B):
        
        try:
            sample_width = BLUE_B['duration_secs']*2000
            subset = x_2k[(t1*2000)-sample_width:(t2*2000)+sample_width]
            
            im, p50, p75, p90 = mel_spectrogram(subset, 2000, BLUE_B['num_fft'], OVERLAP, [25, 75], BLUE_B['duration_secs'], 'blue')
            p = 1
            mid_f = mid_freq(im)
            mid_f = (f2-f1)*(mid_f/224)
            mid_f+=f1
            if (40<mid_f<50):
                
                annotations_B.append([date, hour, t1, 'BLUE_B', p, sample_width, mid_f, p50, p75, p90])
                name = f'{date}T{hour}-{t1}-BlueB'
                plt.imsave(os.path.join(directory_B, name+'.png'), im, origin='lower')

        except:
            pass
            
    print("{} Confirmed Blue-A calls in hour {}".format(len(annotations_A), hour))
    print("{} Possible Blue-B calls in hour {}".format(len(annotations_B), hour))
    return annotations_A + annotations_B

############################################################################################

def predict_humphback(psound_segment_at_10k):
    
    context_step_samples = tf.cast(10000, tf.int64)
    waveform1 = np.expand_dims(psound_segment_at_10k, axis=1)
    waveform_exp = tf.expand_dims(waveform1, 0)
    psound_scores = model_humphback_score(waveform=waveform_exp, context_step_samples=context_step_samples)
    score_values = psound_scores['scores'].numpy()[0]
    return score_values

def humphback_analysis(x_10k, x_2k, hour, date):
    
    annotations = []

    x_brk = np.split(x_10k, 72)
    
    try:
        os.mkdir(r'C:\Users\vikas\OneDrive\Artash_Python\2023 Test\1 - Whale Detection\Annotations\{}'.format(date))
    except:
        pass
    
    directory = r'C:\Users\vikas\OneDrive\Artash_Python\2023 Test\1 - Whale Detection\Annotations\{}\Humphback'.format(date)
    
    try:
        os.mkdir(directory)
    except:
        pass
    
    for i, elem in enumerate(x_brk):
        mean_score = np.mean(predict_humphback(elem))
        if mean_score > 0.7:
            
            subset_2k = x_2k[i*50*2000:(i+1)*50*2000]
            im, p50, p75, p90 = mel_spectrogram(subset_2k, 2000, 1024, OVERLAP, [100, 1000], 50, 'humphback')
            mid_f = mid_freq(im)
            mid_f = 900*(mid_f/224)
            mid_f+=100
            name = f'{date}T{hour}-{(i*50)}-Humphback'
            
            plt.imsave(os.path.join(directory, name+'.png'), im, origin='lower')
            annotations.append([date, hour, (i*50), 'Humpback', mean_score, 50, mid_f, p50, p75, p90])
            
        if mean_score < 0.07:
            
            subset_2k = x_2k[i*50*2000:(i+1)*50*2000]
            im, p50, p75, p90 = mel_spectrogram(subset_2k, 2000, 1024, OVERLAP, [100, 1000], 50, 'humphback')
            name = f'{date}T{hour}-{(i*50)}-NoiseH'
            
            plt.imsave(os.path.join(r'C:\Users\vikas\OneDrive\Artash_Python\2023 Test\1 - Whale Detection\Annotations\Noise', name+'.png'), im, origin='lower')
            
            
    print("{} Confirmed Humpback calls in hour {}".format(len(annotations), hour))
    return annotations
        

----------------
Analyzing Specified Day Hour by Hour

----------------

In [ ]:
labels = []

for start_hour in range(0, 24):
    
    date = wav_filename.split('-')[1].split('T')[0]
    
    start_frame = int(SAMPLE_RATE * start_hour * 3600)
    duration_frames =  int(SAMPLE_RATE* 3600)

    pacsound_file = sf.SoundFile(wav_filename_full)
    pacsound_file.seek(start_frame)
    x = pacsound_file.read(duration_frames, dtype='float32')
    
    print("1 - BLUE WHALE ANALYSIS (A Calls, B Calls)")
    x_2k = scipy.signal.resample(x, (2000*3600))
    x_10k = scipy.signal.resample(x, (10000*3600))
    
    l = BLUE_Analysis(x_2k, start_hour, date)
    labels += l
    print()
    print("2 - HUMPHBACK WHALE ANALYSIS (Songs)")
    l = humphback_analysis(x_10k, x_2k, start_hour, date)
    labels+=l
    print()
    

1 - BLUE WHALE ANALYSIS (A Calls, B Calls)
12 Area-Of-Interests (A-Calls) were found in hour 0
22 Area-Of-Interests (B-Calls) were found in hour 0


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

9 Confirmed Blue-A calls in hour 0
22 Possible Blue-B calls in hour 0

2 - HUMPHBACK WHALE ANALYSIS (Songs)
0 Confirmed Humpback calls in hour 0

1 - BLUE WHALE ANALYSIS (A Calls, B Calls)
11 Area-Of-Interests (A-Calls) were found in hour 1
16 Area-Of-Interests (B-Calls) were found in hour 1


  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

5 Confirmed Blue-A calls in hour 1
16 Possible Blue-B calls in hour 1

2 - HUMPHBACK WHALE ANALYSIS (Songs)


In [1]:
1

1

In [9]:
df = pd.DataFrame(labels, columns= ['Date', 'Hour','Second', 'Label', 'Confidence', 'Duration', 'Mean Frequency', 'L50 Power', 'L75 Power', 'L90 Power'])

In [10]:
df.to_excel(os.path.join(r'C:\Users\vikas\OneDrive\Artash_Python\2023 Test\1 - Whale Detection\Annotations', date+'.xlsx'))